In [1]:
from settings import s, e
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import pickle

from IPython.display import clear_output, display, update_display
import matplotlib.pyplot as plt

clear_output()

# Game Check

# MultiStepBuffer Check

In [3]:
from agent_code.tensor_agent.agent import MultiStepBuffer

from agent_code.tensor_agent.model import FullModel

In [17]:
input_shape = (4,4,1)
D = 6
model = FullModel(input_shape, D)

ms_buffer = MultiStepBuffer()

X = np.zeros((4,4,1))
X[1,2] = 1
action = 2
reward = 1-0.0001
sample = [X, action, reward]
result = ms_buffer.push(*sample, target_model=model.target)
assert(result is None)
result = ms_buffer.push(*sample, target_model=model.target)
assert(result is None)
result = ms_buffer.push(*sample, target_model=model.target)
assert(result is None)

Xs, actions, rewards = ms_buffer.clear()

In [19]:
np.all(Xs[0] == X) and actions[0] == action and rewards[0] == reward

True

In [21]:
result == None

True

# Sum Tree

In [121]:
from agent_code.tensor_agent.per import SumTree

In [132]:
capacity = 6
tree = SumTree(capacity)

In [133]:
for i in range(capacity):
    tree.add(1., 'huhu')

In [129]:
tree.tree

array([1.000e+04, 5.904e+03, 4.096e+03, ..., 1.000e+00, 1.000e+00,
       1.000e+00])

In [75]:
tree.update(capacity-1, 0.5)
tree.total_priority=tree.tree[0]

In [76]:
tree.tree

array([5.5, 4. , 1.5, 2. , 2. , 0.5, 1. , 1. , 1. , 1. , 1. ])

In [77]:
np.sum(tree.tree[-tree.capacity:] != 1)

1

In [130]:

def sample(self, k):  #k:how many experiences in one sample

    priority_range = tree.total_priority/k
    minibatch=[]
    idxs = np.ones((k,), dtype=np.int32)
    weights = np.ones((k, 1))
    #to normalize the weights, the maximal weight needs to be calculated
    max_weight=1/(k*np.min(tree.tree[-tree.capacity:]))**hp.PER_b

    for i in range(k):
        lower_bound = priority_range*i
        upper_bound=lower_bound+priority_range

        #now get a random sample from that range
        value=np.random.uniform(lower_bound,upper_bound)
        leave_index, value_priority, value_data = tree.get_leave(value)
        print(lower_bound, upper_bound, value, value_priority, leave_index)
        idxs[i]=leave_index            
        prob_weight=value_priority/tree.total_priority
        weights[i,0]=(1/(k*prob_weight)**hp.PER_b)/max_weight
        minibatch.append(value_data)
    hp.PER_b=np.minimum(1., hp.PER_b+hp.PER_anneal)
    return idxs, minibatch, weights

In [131]:
sample(None, 32)

0 291.0970497611234
1 291.0970497611234
3 291.0970497611234
7 291.0970497611234
15 291.0970497611234
31 291.0970497611234
64 35.0970497611234
129 35.0970497611234
259 35.0970497611234
520 3.097049761123401
1041 3.097049761123401
2083 3.097049761123401
4167 3.097049761123401
8336 1.097049761123401
0.0 312.5 291.0970497611234 1.0 16674
0 467.60660718716235
1 467.60660718716235
3 467.60660718716235
7 467.60660718716235
15 467.60660718716235
31 467.60660718716235
64 211.60660718716235
130 83.60660718716235
262 19.606607187162354
525 19.606607187162354
1052 3.6066071871623535
2105 3.6066071871623535
4211 3.6066071871623535
8424 1.6066071871623535
312.5 625.0 467.60660718716235 1.0 16850
0 708.3076523759864
1 708.3076523759864
3 708.3076523759864
7 708.3076523759864
15 708.3076523759864
32 196.30765237598644
65 196.30765237598644
132 68.30765237598644
266 4.307652375986436
533 4.307652375986436
1067 4.307652375986436
2135 4.307652375986436
4272 0.3076523759864358
8545 0.3076523759864358
625.

(array([16674, 16850, 17091, 17343, 17684, 17993, 18278, 18746, 18964,
        19433, 19682, 19942, 10324, 10621, 10899, 11178, 11559, 11884,
        12157, 12335, 12637, 13208, 13410, 13597, 14107, 14432, 14725,
        14942, 15161, 15546, 16069, 16362], dtype=int32),
 ['huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu',
  'huhu'],
 array([[39.81929806],
        [39.81929806],
        [39.81929806],
        [39.81929806],
        [39.81929806],
        [39.81929806],
        [39.81929806],
        [39.81929806],
        [39.81929806],
        [39.81929806],
        [39.81929806],
        [39.81929806],
        [39.81929806],
        [39.81929806],
        [39.81929806],
        [39.81929806],
        [39.81929806],
        [39.81929

In [120]:
value = .5
parent=0

while True:

    left_child=parent*2+1
    right_child=left_child+1
    if left_child>=len(tree.tree):
        break

    print(parent, value, '-', left_child, tree.tree[left_child], '-', right_child,tree.tree[right_child])
    if tree.tree[left_child]>=value:
        parent=left_child
    else:
        value-=tree.tree[left_child]
        parent=right_child
leave_index=parent
print(leave_index, tree.data[leave_index-tree.capacity+1])

0 0.5 - 1 4.0 - 2 2.0
1 0.5 - 3 2.0 - 4 2.0
3 0.5 - 7 1.0 - 8 1.0
7 huhu


In [113]:
tree.tree

array([6., 4., 2., 2., 2., 1., 1., 1., 1., 1., 1.])